In [1]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm.auto import tqdm
from fuzzywuzzy import fuzz

In [2]:
res_df = pd.read_parquet("artists_with_relations.parquet")
res_df

,Artist,Spotify Name,Genres,Popularity,Spotify ID,followers,influencers,searched_artist_name
0,Motörhead,Motörhead,"[album rock, hard rock, metal, rock, speed metal]",68.0,1DFr97A9HnbV3SKTJFu62M,"[Alice in Chains, Ween, Iron Maiden, Pantera, ...","[Sex Pistols, The Yardbirds, Ramones, The Stoo...",Motörhead
1,Color Me Badd,Color Me Badd,"[boy band, contemporary r&b, new jack swing, r...",53.0,1QtIfAa6y7w2JhxYJhYeUG,[],"[New Edition, Jodeci, Guy]",Color Me Badd
2,Trumans Water,Trumans Water,[noise rock],7.0,3GQRTA9DtNlGbQqu8ST1Xx,"[No Age, The Fiery Furnaces]","[Sonic Youth, The Plastic People of the Univer...",Trumans Water
3,Cliff Eberhardt,Cliff Eberhardt,[contemporary folk],10.0,1LpAlKD69cCn6ygqov2bGs,[],[Steve Forbert],Cliff Eberhardt
4,The Waitresses,The Waitresses,[new wave pop],40.0,57AKOY1SF1062tuVzgAPv8,[],"[Blondie, Battant]",The Waitresses
...,...,...,...,...,...,...,...,...
10111,Wumpscut,:Wumpscut:,"[aggrotech, dark wave, ebm, electro-industrial...",37.0,3TTbkikkCAbUg2j5i9UrTV,[],[Skinny Puppy],:Wumpscut:
10112,54-40,54-40,"[canadian indie, canadian rock]",38.0,1Y4l4YawNAZndGBSrZxb3N,[Hootie & The Blowfish],[],54-40
10113,10_000 Maniacs,"10,000 Maniacs","[folk, lilith, mellow gold, new wave pop, pop ...",53.0,0MBIKH9DjtBkv8O3nS6szj,"[Downy Mildew, Ivy, Jewel, Joss Stone, Lisa Lo...","[The Byrds, Elvis Costello, Cat Stevens, Fleet...","10,000 Maniacs"
10114,...And You Will Know Us by the Trail of Dead,...And You Will Know Us by the Trail of Dead,"[alternative rock, double drumming, indie rock...",41.0,6wBUn8gMP85n8dPu6LoUcF,[Oxford Collapse],"[Sonic Youth, Fugazi, Jane's Addiction, Drive ...",...And You Will Know Us by the Trail of Dead


In [3]:
def fuzzy_similarity(row):
    sim1 = fuzz.token_sort_ratio(row['Artist'], row['searched_artist_name'])
    sim2 = fuzz.token_sort_ratio(row['Spotify Name'], row['searched_artist_name'])
    return max(sim1, sim2)

res_df['Similarity'] = res_df.apply(fuzzy_similarity, axis=1)
res_df

,Artist,Spotify Name,Genres,Popularity,Spotify ID,followers,influencers,searched_artist_name,Similarity
0,Motörhead,Motörhead,"[album rock, hard rock, metal, rock, speed metal]",68.0,1DFr97A9HnbV3SKTJFu62M,"[Alice in Chains, Ween, Iron Maiden, Pantera, ...","[Sex Pistols, The Yardbirds, Ramones, The Stoo...",Motörhead,100
1,Color Me Badd,Color Me Badd,"[boy band, contemporary r&b, new jack swing, r...",53.0,1QtIfAa6y7w2JhxYJhYeUG,[],"[New Edition, Jodeci, Guy]",Color Me Badd,100
2,Trumans Water,Trumans Water,[noise rock],7.0,3GQRTA9DtNlGbQqu8ST1Xx,"[No Age, The Fiery Furnaces]","[Sonic Youth, The Plastic People of the Univer...",Trumans Water,100
3,Cliff Eberhardt,Cliff Eberhardt,[contemporary folk],10.0,1LpAlKD69cCn6ygqov2bGs,[],[Steve Forbert],Cliff Eberhardt,100
4,The Waitresses,The Waitresses,[new wave pop],40.0,57AKOY1SF1062tuVzgAPv8,[],"[Blondie, Battant]",The Waitresses,100
...,...,...,...,...,...,...,...,...,...
10111,Wumpscut,:Wumpscut:,"[aggrotech, dark wave, ebm, electro-industrial...",37.0,3TTbkikkCAbUg2j5i9UrTV,[],[Skinny Puppy],:Wumpscut:,100
10112,54-40,54-40,"[canadian indie, canadian rock]",38.0,1Y4l4YawNAZndGBSrZxb3N,[Hootie & The Blowfish],[],54-40,100
10113,10_000 Maniacs,"10,000 Maniacs","[folk, lilith, mellow gold, new wave pop, pop ...",53.0,0MBIKH9DjtBkv8O3nS6szj,"[Downy Mildew, Ivy, Jewel, Joss Stone, Lisa Lo...","[The Byrds, Elvis Costello, Cat Stevens, Fleet...","10,000 Maniacs",100
10114,...And You Will Know Us by the Trail of Dead,...And You Will Know Us by the Trail of Dead,"[alternative rock, double drumming, indie rock...",41.0,6wBUn8gMP85n8dPu6LoUcF,[Oxford Collapse],"[Sonic Youth, Fugazi, Jane's Addiction, Drive ...",...And You Will Know Us by the Trail of Dead,100


In [4]:
res_df_leftover = res_df[res_df['Similarity'] < 95]
res_df_leftover

,Artist,Spotify Name,Genres,Popularity,Spotify ID,followers,influencers,searched_artist_name,Similarity
62,Funk Brothers,Funk Brothers,None,2.0,6HcTUAkHBgVHAB9d5rr6jM,"[Jack Bruce, Vulfpeck, Sly & Robbie]",[],The Funk Brothers,87
105,Partner,Partner,[new brunswick indie],22.0,15rbZYMoW9AYb3zcuFUNOU,[],[],Partners In Kryme,58
134,Dj Esco,DJ ESCO,"[atl hip hop, rap, southern hip hop, trap, vap...",49.0,2J0JN7EFN10G1Tty6hX0AN,[],[],DJ ESCOBAR,82
152,MJG,MJG,"[dirty south rap, gangster rap, memphis hip ho...",46.0,6pDFdYZARPitpkfm44UsOQ,"[Kanye West, Gucci Mane, Mystikal, Lil Jon, Th...","[N.W.A, Run-D.M.C.]",8Ball & MJG,50
178,Zapp,Zapp,"[funk, gangster rap, p funk, popping, post-dis...",55.0,396Kh0m4wGUvcMUULw71yi,"[Devo, Alice Cooper, Butthole Surfers, Dweezil...","[John Coltrane, Clarence ""Gatemouth"" Brown, Ed...",Frank Zappa,53
...,...,...,...,...,...,...,...,...,...
9806,Ike And Tina Turner,Ike & Tina Turner,"[classic soul, funk, soul, southern soul]",56.0,1ZikppG9dPedbIgMfnfx8k,[],[],Ike Parodi,48
9847,Georgie Fame & The Blue Flames,Georgie Fame & The Blue Flames,"[british blues, british invasion, merseybeat]",35.0,0mopUs7SL5szuPOcOsS73A,"[John McLaughlin, Elvis Costello, The Zombies,...","[Ray Charles, Jimmy Smith, Lee Dorsey, George ...",Georgie Fame,60
9873,Frankie Lymon & The Teenagers_ Lymon_ Levy,Frankie Lymon,"[doo-wop, rhythm and blues]",43.0,2KMQz6s6MNRr4bxI8EQdRk,[],[],Frankie Lymon & The Teenagers,81
10014,Chick Corea / Béla Fleck,Béla Fleck,"[banjo, bluegrass, instrumental bluegrass, jam...",46.0,2ka8z2lwkcp13fG8Wyv3xU,[],[],Béla Fleck and the Flecktones,49


In [5]:
import webbrowser

def filter_rows(df):
    filtered_rows = []
    for index, row in df.iterrows():
        print(f"Artist: {row['Artist']}, Spotify Name: {row['Spotify Name']}, Searched Artist Name: {row['searched_artist_name']}")
        user_input = input("Include this row? (T/F/C)")
        while user_input:
            if user_input.lower() == 't':
                filtered_rows.append(row)
                print(f"Appended {row['searched_artist_name']}")
                break
            elif user_input.lower() == 'c':
                id = row['Spotify ID']
                name = row['searched_artist_name']
                webbrowser.open_new_tab(f'https://open.spotify.com/artist/{id}')
                webbrowser.open_new_tab(f'https://www.google.com/search?q={name}')
                user_input = input("Really include this row? (T/F/C)")
                if user_input.lower() == 't':
                    filtered_rows.append(row)
                    print(f"Appended {row['searched_artist_name']}")
                    break
                else:
                    print(print(f"Skipped {row['searched_artist_name']}"))
                    break
            else:
                print(f"Skipped {row['searched_artist_name']}")
                break
    return pd.DataFrame(filtered_rows)

res_df_leftover_pickup = filter_rows(res_df_leftover)
res_df_leftover_pickup

Artist: Funk Brothers, Spotify Name: Funk Brothers, Searched Artist Name: The Funk Brothers
Appended The Funk Brothers
Artist: Partner, Spotify Name: Partner, Searched Artist Name: Partners In Kryme
Skipped Partners In Kryme
Artist: Dj Esco, Spotify Name: DJ ESCO, Searched Artist Name: DJ ESCOBAR
Skipped DJ ESCOBAR
Artist: MJG, Spotify Name: MJG, Searched Artist Name: 8Ball & MJG
Appended 8Ball & MJG
Artist: Zapp, Spotify Name: Zapp, Searched Artist Name: Frank Zappa
Appended Frank Zappa
Artist: Cascada, Spotify Name: Cascada, Searched Artist Name: Sonidos de Cascadas
Skipped Sonidos de Cascadas
None
Artist: Pell, Spotify Name: Pell, Searched Artist Name: PelleK
Skipped PelleK
Artist: Mission, Spotify Name: Mission, Searched Artist Name: The Mission
Skipped The Mission
None
Artist: Richard X, Spotify Name: Richard x, Searched Artist Name: Richard Clayderman
Skipped Richard Clayderman
None
Artist: Jonathan Richman And The Modern Lovers, Spotify Name: Jonathan Richman & The Modern Lovers

,Artist,Spotify Name,Genres,Popularity,Spotify ID,followers,influencers,searched_artist_name,Similarity
62,Funk Brothers,Funk Brothers,None,2.0,6HcTUAkHBgVHAB9d5rr6jM,"[Jack Bruce, Vulfpeck, Sly & Robbie]",[],The Funk Brothers,87
152,MJG,MJG,"[dirty south rap, gangster rap, memphis hip ho...",46.0,6pDFdYZARPitpkfm44UsOQ,"[Kanye West, Gucci Mane, Mystikal, Lil Jon, Th...","[N.W.A, Run-D.M.C.]",8Ball & MJG,50
178,Zapp,Zapp,"[funk, gangster rap, p funk, popping, post-dis...",55.0,396Kh0m4wGUvcMUULw71yi,"[Devo, Alice Cooper, Butthole Surfers, Dweezil...","[John Coltrane, Clarence ""Gatemouth"" Brown, Ed...",Frank Zappa,53
397,Jonathan Richman And The Modern Lovers,Jonathan Richman & The Modern Lovers,[boston rock],40.0,6hkch2KhRl0tywpeVK5xR5,"[Talking Heads, Ramones, Daniel Johnston, The ...","[The Beatles, The Velvet Underground, Buddy Ho...",Jonathan Richman,64
478,Dave Ellis,Dave Ellis,None,3.0,4FHelmK7qDIv5fWF6bxdsr,[],[],Dave Ellis & Boo Howard,65
807,Triple 6,Triple 6,None,5.0,4XJt5LyIAoE745jCaVIqml,"[Gucci Mane, Young Jeezy, Lil Jon, Bobby Raps,...","[N.W.A, Master P, 8Ball & MJG, Geto Boys, Dr. ...",Three 6 Mafia,48
1006,Cecil Sharp,Cecil Sharp,None,1.0,4ZjhrdLNlFwydWlbYAVYXM,[],[],Cecil Sharp Project,73
1125,Mýa,Mýa,"[contemporary r&b, dance pop, hip pop, neo sou...",65.0,6lHL3ubAMgSasKjNqKb8HF,[],[],MYA,80
1357,Rance Allen,Rance Allen,[gospel soul],32.0,6kSjbNXcS3bNyP3WSli67X,[],[],The Rance Allen Group,69
1597,Billy Ward,Billy Ward,None,30.0,3H2Q9YgssgbkLzFT1HGJlp,[Ben E. King],[],Billy Ward & The Dominoes,61


In [9]:
res_df_leftover_pickup

,Artist,Spotify Name,Genres,Popularity,Spotify ID,followers,influencers,searched_artist_name,Similarity
62,Funk Brothers,Funk Brothers,None,2.0,6HcTUAkHBgVHAB9d5rr6jM,"[Jack Bruce, Vulfpeck, Sly & Robbie]",[],The Funk Brothers,87
152,MJG,MJG,"[dirty south rap, gangster rap, memphis hip ho...",46.0,6pDFdYZARPitpkfm44UsOQ,"[Kanye West, Gucci Mane, Mystikal, Lil Jon, Th...","[N.W.A, Run-D.M.C.]",8Ball & MJG,50
178,Zapp,Zapp,"[funk, gangster rap, p funk, popping, post-dis...",55.0,396Kh0m4wGUvcMUULw71yi,"[Devo, Alice Cooper, Butthole Surfers, Dweezil...","[John Coltrane, Clarence ""Gatemouth"" Brown, Ed...",Frank Zappa,53
397,Jonathan Richman And The Modern Lovers,Jonathan Richman & The Modern Lovers,[boston rock],40.0,6hkch2KhRl0tywpeVK5xR5,"[Talking Heads, Ramones, Daniel Johnston, The ...","[The Beatles, The Velvet Underground, Buddy Ho...",Jonathan Richman,64
478,Dave Ellis,Dave Ellis,None,3.0,4FHelmK7qDIv5fWF6bxdsr,[],[],Dave Ellis & Boo Howard,65
807,Triple 6,Triple 6,None,5.0,4XJt5LyIAoE745jCaVIqml,"[Gucci Mane, Young Jeezy, Lil Jon, Bobby Raps,...","[N.W.A, Master P, 8Ball & MJG, Geto Boys, Dr. ...",Three 6 Mafia,48
1006,Cecil Sharp,Cecil Sharp,None,1.0,4ZjhrdLNlFwydWlbYAVYXM,[],[],Cecil Sharp Project,73
1125,Mýa,Mýa,"[contemporary r&b, dance pop, hip pop, neo sou...",65.0,6lHL3ubAMgSasKjNqKb8HF,[],[],MYA,80
1357,Rance Allen,Rance Allen,[gospel soul],32.0,6kSjbNXcS3bNyP3WSli67X,[],[],The Rance Allen Group,69
1597,Billy Ward,Billy Ward,None,30.0,3H2Q9YgssgbkLzFT1HGJlp,[Ben E. King],[],Billy Ward & The Dominoes,61


In [12]:
pickup = res_df_leftover_pickup
pickup = pickup.drop('Similarity', axis=1)
pickup

,Artist,Spotify Name,Genres,Popularity,Spotify ID,followers,influencers,searched_artist_name
62,Funk Brothers,Funk Brothers,None,2.0,6HcTUAkHBgVHAB9d5rr6jM,"[Jack Bruce, Vulfpeck, Sly & Robbie]",[],The Funk Brothers
152,MJG,MJG,"[dirty south rap, gangster rap, memphis hip ho...",46.0,6pDFdYZARPitpkfm44UsOQ,"[Kanye West, Gucci Mane, Mystikal, Lil Jon, Th...","[N.W.A, Run-D.M.C.]",8Ball & MJG
178,Zapp,Zapp,"[funk, gangster rap, p funk, popping, post-dis...",55.0,396Kh0m4wGUvcMUULw71yi,"[Devo, Alice Cooper, Butthole Surfers, Dweezil...","[John Coltrane, Clarence ""Gatemouth"" Brown, Ed...",Frank Zappa
397,Jonathan Richman And The Modern Lovers,Jonathan Richman & The Modern Lovers,[boston rock],40.0,6hkch2KhRl0tywpeVK5xR5,"[Talking Heads, Ramones, Daniel Johnston, The ...","[The Beatles, The Velvet Underground, Buddy Ho...",Jonathan Richman
478,Dave Ellis,Dave Ellis,None,3.0,4FHelmK7qDIv5fWF6bxdsr,[],[],Dave Ellis & Boo Howard
807,Triple 6,Triple 6,None,5.0,4XJt5LyIAoE745jCaVIqml,"[Gucci Mane, Young Jeezy, Lil Jon, Bobby Raps,...","[N.W.A, Master P, 8Ball & MJG, Geto Boys, Dr. ...",Three 6 Mafia
1006,Cecil Sharp,Cecil Sharp,None,1.0,4ZjhrdLNlFwydWlbYAVYXM,[],[],Cecil Sharp Project
1125,Mýa,Mýa,"[contemporary r&b, dance pop, hip pop, neo sou...",65.0,6lHL3ubAMgSasKjNqKb8HF,[],[],MYA
1357,Rance Allen,Rance Allen,[gospel soul],32.0,6kSjbNXcS3bNyP3WSli67X,[],[],The Rance Allen Group
1597,Billy Ward,Billy Ward,None,30.0,3H2Q9YgssgbkLzFT1HGJlp,[Ben E. King],[],Billy Ward & The Dominoes


In [13]:
a = res_df[res_df['Similarity'] >= 95]
fin_df = pd.concat([a, pickup], axis=0)
fin_df

,Artist,Spotify Name,Genres,Popularity,Spotify ID,followers,influencers,searched_artist_name,Similarity
0,Motörhead,Motörhead,"[album rock, hard rock, metal, rock, speed metal]",68.0,1DFr97A9HnbV3SKTJFu62M,"[Alice in Chains, Ween, Iron Maiden, Pantera, ...","[Sex Pistols, The Yardbirds, Ramones, The Stoo...",Motörhead,100.0
1,Color Me Badd,Color Me Badd,"[boy band, contemporary r&b, new jack swing, r...",53.0,1QtIfAa6y7w2JhxYJhYeUG,[],"[New Edition, Jodeci, Guy]",Color Me Badd,100.0
2,Trumans Water,Trumans Water,[noise rock],7.0,3GQRTA9DtNlGbQqu8ST1Xx,"[No Age, The Fiery Furnaces]","[Sonic Youth, The Plastic People of the Univer...",Trumans Water,100.0
3,Cliff Eberhardt,Cliff Eberhardt,[contemporary folk],10.0,1LpAlKD69cCn6ygqov2bGs,[],[Steve Forbert],Cliff Eberhardt,100.0
4,The Waitresses,The Waitresses,[new wave pop],40.0,57AKOY1SF1062tuVzgAPv8,[],"[Blondie, Battant]",The Waitresses,100.0
...,...,...,...,...,...,...,...,...,...
9184,Louis Armstrong - Itinéraire d'un génie,Louis Armstrong & His Hot Five,"[hot jazz, new orleans jazz]",31.0,19eLuQmk9aCobbVDHc6eek,"[George Benson, Miles Davis, Charlie Parker, D...","[King Oliver, Bunk Johnson, Kid Ory, Fletcher ...",Louis Armstrong,NaN
9454,Steve Harley,Steve Harley,"[classic uk pop, glam rock, pub rock]",50.0,3dS8rLINyM7EYuMXryXJym,[],[The Pretty Things],Steve Harley & Cockney Rebel,NaN
9847,Georgie Fame & The Blue Flames,Georgie Fame & The Blue Flames,"[british blues, british invasion, merseybeat]",35.0,0mopUs7SL5szuPOcOsS73A,"[John McLaughlin, Elvis Costello, The Zombies,...","[Ray Charles, Jimmy Smith, Lee Dorsey, George ...",Georgie Fame,NaN
9873,Frankie Lymon & The Teenagers_ Lymon_ Levy,Frankie Lymon,"[doo-wop, rhythm and blues]",43.0,2KMQz6s6MNRr4bxI8EQdRk,[],[],Frankie Lymon & The Teenagers,NaN


In [17]:
finres_df = fin_df[(fin_df['followers'].str.len() != 0) | (fin_df['influencers'].str.len() != 0)]
finres_df = finres_df.drop('Similarity', axis=1)
finres_df

,Artist,Spotify Name,Genres,Popularity,Spotify ID,followers,influencers,searched_artist_name
0,Motörhead,Motörhead,"[album rock, hard rock, metal, rock, speed metal]",68.0,1DFr97A9HnbV3SKTJFu62M,"[Alice in Chains, Ween, Iron Maiden, Pantera, ...","[Sex Pistols, The Yardbirds, Ramones, The Stoo...",Motörhead
1,Color Me Badd,Color Me Badd,"[boy band, contemporary r&b, new jack swing, r...",53.0,1QtIfAa6y7w2JhxYJhYeUG,[],"[New Edition, Jodeci, Guy]",Color Me Badd
2,Trumans Water,Trumans Water,[noise rock],7.0,3GQRTA9DtNlGbQqu8ST1Xx,"[No Age, The Fiery Furnaces]","[Sonic Youth, The Plastic People of the Univer...",Trumans Water
3,Cliff Eberhardt,Cliff Eberhardt,[contemporary folk],10.0,1LpAlKD69cCn6ygqov2bGs,[],[Steve Forbert],Cliff Eberhardt
4,The Waitresses,The Waitresses,[new wave pop],40.0,57AKOY1SF1062tuVzgAPv8,[],"[Blondie, Battant]",The Waitresses
...,...,...,...,...,...,...,...,...
9117,Diana Ross and the Supremes,Diana Ross & The Supremes,"[classic girl group, motown, soul]",51.0,3MdG05syQeRYPPcClLaUGl,"[Beyoncé, Mary J. Blige, Brenda Holloway, Donn...","[The Temptations, Billie Holiday, Ella Fitzger...",Diana Ross
9128,Frank Black Francis,Frank Black & The Catholics,[alternative rock],30.0,5Q8gM5SMBT2LAoZ5LzqAvH,"[Eddie Vedder, Dave Grohl, Veruca Salt, Superc...","[The Beach Boys, David Bowie, Hüsker Dü, Iggy ...",Frank Black
9184,Louis Armstrong - Itinéraire d'un génie,Louis Armstrong & His Hot Five,"[hot jazz, new orleans jazz]",31.0,19eLuQmk9aCobbVDHc6eek,"[George Benson, Miles Davis, Charlie Parker, D...","[King Oliver, Bunk Johnson, Kid Ory, Fletcher ...",Louis Armstrong
9454,Steve Harley,Steve Harley,"[classic uk pop, glam rock, pub rock]",50.0,3dS8rLINyM7EYuMXryXJym,[],[The Pretty Things],Steve Harley & Cockney Rebel


In [18]:
finres_df.to_parquet('filtered_artists_with_relations.parquet')